In [ ]:
!pip install -q transformers scikit-learn pandas torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# to import files from googledrive
from google.colab import drive


drive.mount('/content/gdrive/', force_remount=True)
sample_size_dir = 'gdrive/My Drive/Sample size for NLP'

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv(sample_size_dir+'/data_jaya/51881_combined_data_1000.csv')
df.head()

In [ ]:
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext') # bert-base-uncased, cambridgeltl/SapBERT-from-PubMedBERT-fulltext
model = BertModel.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext') # bert-base-uncased, cambridgeltl/SapBERT-from-PubMedBERT-fulltext

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
# Function to calculate cosine similarity between two embeddings
def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

In [ ]:
# Sentence to compare with
comparison_sentence = "This patient has been diagnosed with acute respiratory failure"

In [ ]:
# Get BERT embedding for the comparison sentence
comparison_embedding = get_bert_embeddings(comparison_sentence)

In [ ]:
# Calculate cosine similarity for each row and add as a new column
df['cosine score'] = df['TEXT'].apply(lambda x: calculate_cosine_similarity(get_bert_embeddings(x), comparison_embedding))


In [ ]:
df.tail()

In [ ]:
df.to_csv(sample_size_dir+'/data_jaya/51881_combined_data_1000_w_cosine_sapbert.csv')

# word2vec

In [ ]:
!pip install -q gensim scikit-learn


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
df = pd.read_csv(sample_size_dir+'/data_jaya/51881_combined_data_1000.csv')
df.head()

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    return text.lower().split()

In [ ]:
# Prepare sentences for Word2Vec training
sentences = df['TEXT'].apply(preprocess_text).tolist()

In [ ]:
# Train a Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Function to get Word2Vec embeddings for a sentence
def get_word2vec_embeddings(sentence):
    words = preprocess_text(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

In [ ]:
# Sentence to compare with
comparison_sentence = "This patient has been diagnosed with acute respiratory failure"

In [ ]:
# Get Word2Vec embedding for the comparison sentence
comparison_embedding = get_word2vec_embeddings(comparison_sentence)

In [ ]:
# Calculate cosine similarity for each row and add as a new column
df['cosine score'] = df['TEXT'].apply(lambda x: cosine_similarity([get_word2vec_embeddings(x)], [comparison_embedding])[0][0])

In [ ]:
df.head()


In [ ]:
df.to_csv(sample_size_dir+'/data_jaya/51881_combined_data_1000_w_cosine_w2v.csv')